In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import theano
import theano.tensor as T
import crbm as C
import time

In [2]:
#split the dataset in traininig,validation and test set
def splitDataset(dt):
    y = dt.year
    if(y>=2006 and y<=2008):
        return 'training'
    if(y==2009):
        return 'validation'
    if(y==2010):
        return 'test'

#remove the null rows
def removeNullRows(dataSet):
    idxNAN = pd.isnull(dataSet).any(1).nonzero()[0]

#since there are the Nan, we should remove it before the training
#therefore, we split the traiing set as sequnces of series without Nan
    start = 0
    idxSequences = []
    seqlen = []
    for idx in idxNAN:
        if(start < idx):
            #print str(start) + '-' + str(idx-1)
            idxSequences += range(start,idx)
            seqlen += [idx-start]
            start = idx+1
        else:
            start = start +1
    #print str(start) + '-' + str(len(dataSet))
    idxSequences += range(start,len(dataSet))
    seqlen +=  [len(dataSet)-start]
    #print idxSequences
    return dataSet.iloc[idxSequences],seqlen

#normalize the values
def normalizeValues(dataSet):
    return (dataSet - dataSet.mean())/ dataSet.std()


In [3]:
def build_history(data_idx_vec):
    hist_idx = np.zeros((len(data_idx_vec),24),dtype = np.int32)
    for k in range(len(data_idx_vec)):
        count = 0
        data_idx = data_idx_vec[k]
        for i in range(7,0,-1):
            for j in [-1,0,1]:
                hist_idx[k,count] = data_idx - 24*i + j
                count=count+1
        for j in range(3,0,-1):
            hist_idx[k,count] = data_idx-j
            count = count +1
    return hist_idx

In [4]:
def my_training(trainingSet_matrix,seqlenTR,n_hidden = 5,batch_size=24,training_epochs=200):

    print '\nN_HIDDEN='+str(n_hidden)
    
    #we fix the delay in 24 variable. They are:
    # t-1 t t+1 in the 7 previous day
    # t-3 t-2 t-1 in the actual day
    delay = 21+3
    
    #learning rate
    learning_rate = 0.001;
    
    # compute number of visible units
    n_dim = trainingSet_matrix.shape[1]
    
    #create the shared variable for the training set
    batchdata = theano.shared(np.asarray(trainingSet_matrix, dtype=theano.config.floatX))

    # allocate symbolic variables for the data
    index = T.lvector()    # index to a [mini]batch
    index_hist = T.lvector()  # index to history
    x = T.matrix('x')  # the data
    x_history = T.matrix('x_history')

    #theano.config.compute_test_value='warn'
    #x.tag.test_value = np.random.randn(batch_size, n_dim)
    #x_history.tag.test_value = np.random.randn(batch_size, n_dim*delay)

    # initialize storage for the persistent chain
    # (state = hidden layer of chain)

    # construct the CRBM class
    crbm = C.CRBM(input=x, input_history=x_history, n_visible=n_dim, n_hidden=n_hidden, delay=delay)

    # get the cost and the gradient corresponding to one step of CD-15
    cost, updates = crbm.get_cost_updates(lr=learning_rate, k=1)

    #we should skip the first week
    h_in_week = 7*24+1
    batchdataindex = []
    last = 0
    for s in seqlenTR:
        batchdataindex += range(last + h_in_week, last + s)
        last += s
    permindex = np.array(batchdataindex)
    n_train_batches = len(permindex)/ batch_size

    train_crbm = theano.function([index, index_hist], cost,
               updates=updates,
               givens={
                        x: batchdata[index],
                        x_history: batchdata[index_hist].reshape((batch_size, delay * n_dim))
                      },
               name='train_crbm')

    plotting_time = 0.
    start_time = time.clock()

    # go through training epochs
    for epoch in xrange(training_epochs):

        # go through the training set
        mean_cost = []
        for batch_index in xrange(n_train_batches):
            #print '\n'
            # indexing is slightly complicated
            # build a linear index to the starting frames for this batch
            # (i.e. time t) gives a batch_size length array for data
            data_idx = permindex[batch_index * batch_size:(batch_index + 1)* batch_size]
            #print batch_index
            #print data_idx
            # now build a linear index to the frames at each delay tap
            
            #hist_idx = np.array([data_idx - n for n in xrange(1, delay + 1)]).T
            hist_idx = build_history(data_idx)
            #print hist_idx
            this_cost = train_crbm(data_idx, hist_idx.ravel())
            #print batch_index, this_cost
            mean_cost += [this_cost]

        print '\rTraining epoch %d, cost is ' % epoch, np.mean(mean_cost),

    end_time = time.clock()

    pretraining_time = (end_time - start_time)

    print ('\nTraining took %f minutes' % (pretraining_time / 60.))
    
    return crbm

In [5]:
#validate on the whole validation set
def my_evaluation(crbm,evaluationSet_matrix,seqlenEVAL,doPlot=False,start=0,end=100):
    n_samples=1
    delay = crbm.delay
    
    h_in_week = 7*24+1
    data_idx = []
    last = 0
    for s in seqlenEVAL:
        data_idx += range(last + h_in_week, last + s)
        last += s
        
    data_idx = np.asarray(data_idx)
    orig_data = np.asarray(evaluationSet_matrix[data_idx],dtype=theano.config.floatX)


    hist_idx = build_history(data_idx)
    
    hist_idx = hist_idx.ravel()
    

    orig_history = np.asarray(evaluationSet_matrix[hist_idx].reshape((len(data_idx), crbm.delay * crbm.n_visible)),dtype=theano.config.floatX)

    generated_series = crbm.generate(orig_data, orig_history, n_samples=n_samples,n_gibbs=30)

    MSE=[None]*crbm.n_visible
    SMAPE=[None]*crbm.n_visible
    for i in range(crbm.n_visible):
        plotGEN = generated_series[:,n_samples-1,i]
        if(doPlot):
            plt.subplot(crbm.n_visible, 1, i+1)
            plt.plot(plotGEN[start:end])
            plt.plot(evaluationSet_matrix[start:end,i])  

        MSE[i] = np.sum(np.power(plotGEN - orig_data[:,i],2))/(len(orig_data))
        SMAPE[i] = np.sum(np.abs(plotGEN - orig_data[:,i]) / (np.abs(plotGEN) + np.abs(orig_data[:,i]))) / len(orig_data) *100                                                                                               
    plt.show()
    return MSE,SMAPE

In [6]:
def addSeasonality(dataSet):
    #day and month of the batchdata
    dataSet_matrix = dataSet.values
    
    idx_train = dataSet.index
    dow = idx_train.dayofweek
    m = idx_train.month
    h = idx_train.hour
    d = idx_train.dayofyear

    season_year = np.cos(((24 * (d-1) + h)*2*np.pi/(365*24-1))+3*np.pi/2)
    season_week = np.cos((dow-1)*2*np.pi/6)
    season_day = np.cos(h*2*np.pi/23)

    #now create a matrix s.t. the column are seasonYear | seasonWeek | seasonDay | allOtherData
    dataSet_matrix = np.column_stack((season_day,dataSet_matrix))
    dataSet_matrix = np.column_stack((season_week,dataSet_matrix))
    dataSet_matrix = np.column_stack((season_year,dataSet_matrix))
    return dataSet_matrix

In [7]:
#load data from file
allData = pd.read_csv('../household_power_consumption.txt',';',index_col=0,na_values='?',header=0,parse_dates=[[0, 1]],infer_datetime_format=True)

allData.dtypes

Global_active_power      float64
Global_reactive_power    float64
Voltage                  float64
Global_intensity         float64
Sub_metering_1           float64
Sub_metering_2           float64
Sub_metering_3           float64
dtype: object

In [8]:
#reduce the number of data coputing the max of each hour
groupedByH = allData.groupby(pd.TimeGrouper('H')).max()
#groupedByH

In [9]:
splittedDataset = groupedByH.groupby(splitDataset)

#split dataset
trainingSet = splittedDataset.get_group('training')
validationSet = splittedDataset.get_group('validation')
testSet = splittedDataset.get_group('test')

#remove null values
trainingSet,seqlenTR = removeNullRows(trainingSet)
validationSet,seqlenVAL = removeNullRows(validationSet)
testSet,seqlenTE = removeNullRows(testSet)

#normaliza all values with 0 mean and 1 std. dev.
trainingSet = normalizeValues(trainingSet)
validationSet = normalizeValues(validationSet)
testSet = normalizeValues(testSet)

In [10]:
#data for the training
trainingSet_matrix = trainingSet.values

# compute number of visible units
n_dim = trainingSet_matrix.shape[1]

In [11]:
#build validation set

validationSet_matrix = validationSet.values

In [12]:
#choose the best setting
n_hidden_values = [3,5,7,10,20,50,100]

store_MSE = np.zeros((len(n_hidden_values),n_dim))
store_SMAPE = np.zeros_like(store_MSE)
store_crbm_Model = [None for i in range(len(n_hidden_values))]
for idx_nh in range(len(n_hidden_values)):
    nh = n_hidden_values[idx_nh]
    crbmMdl = my_training(trainingSet_matrix,seqlenTR,n_hidden=nh,batch_size=24,training_epochs=1000)
    MSE,SMAPE = my_evaluation(crbmMdl,validationSet_matrix,seqlenVAL)

    store_crbm_Model[idx_nh] = crbmMdl
    store_MSE[idx_nh,:] = MSE
    store_SMAPE[idx_nh,:] = SMAPE



N_HIDDEN=3
Training epoch 999, cost is  2.10875647406 
Training took 24.017207 minutes
Generating frame 0

N_HIDDEN=5
Training epoch 999, cost is  1.80263074746 
Training took 24.224040 minutes
Generating frame 0

N_HIDDEN=7
Training epoch 999, cost is  1.96598852487 
Training took 20.118550 minutes
Generating frame 0

N_HIDDEN=10
Training epoch 999, cost is  2.03142605122 
Training took 24.629604 minutes
Generating frame 0

N_HIDDEN=20
Training epoch 999, cost is  2.13788273875 
Training took 25.844320 minutes
Generating frame 0

N_HIDDEN=50
Training epoch 999, cost is  2.22804198592 
Training took 32.793645 minutes
Generating frame 0

N_HIDDEN=100
Training epoch 999, cost is  2.30530199352 
Training took 37.830083 minutes
Generating frame 0


crbm.py:316: UserWarning: Updating an `OrderedUpdates` with a non-ordered dictionary with 2+ elements could make your code non-deterministic
  ), axis=1)})


In [13]:
best_idx_nh = store_MSE.mean(axis=1).argmin()
best_nh_MSE = n_hidden_values[best_idx_nh]
print 'BEST N HIDDEN = '+str(best_nh_MSE)
print 'BEST MSE = '+str(store_MSE[best_idx_nh,:].mean())

BEST N HIDDEN = 3
BEST MSE = 0.75025792574


In [14]:
best_idx_nh = store_SMAPE.mean(axis=1).argmin()
best_nh_SMAPE = n_hidden_values[best_idx_nh]
print 'BEST N HIDDEN = '+str(best_nh_SMAPE)
print 'BEST SMAPE = '+str(store_SMAPE[best_idx_nh,:].mean())

BEST N HIDDEN = 5
BEST SMAPE = 42.0938964663


In [15]:
#concatanate training and validation to obtain a bigger valid
trainingSet_BIG_matrix = np.concatenate((trainingSet_matrix,validationSet_matrix))
seqlenTR_BIG = seqlenTR + seqlenVAL

best_crbm = my_training(trainingSet_BIG_matrix,seqlenTR_BIG,n_hidden=best_nh_MSE,batch_size=2,training_epochs=1000)


N_HIDDEN=3
Training epoch 999, cost is  2.11124506385 
Training took 316.102039 minutes


In [16]:
#build test set
testSet_matrix = testSet.values

best_MSE,best_SMAPE = my_evaluation(best_crbm,testSet_matrix,seqlenTE,doPlot=True)

Generating frame 0


In [17]:
best_MSE

[0.54236776282169941,
 0.91522123667161859,
 0.2936655395071277,
 0.54412603005524407,
 0.59715155318611968,
 0.36573225249480418,
 0.97884942771533767]

In [18]:
best_SMAPE

[47.023793136672268,
 66.098769009894738,
 43.446789822545782,
 47.019433981945085,
 10.211582603925882,
 21.444027053454139,
 53.500950504805743]